# Additive Factors Model
(Based on Cen 2009)
This model is used to explain the power law in learning.  In this notebook we try to build a neuralised version of the AFM and train it using simulated data.  The aim of using the AFM is to disentangle the latent traits that make up the overall score going into the sigmoid probability estimator.

The model is compensatory, which is a weakness.

In [2]:
from collections import defaultdict
from copy import copy
from math import exp, sqrt
from random import random, shuffle, choice, randint, uniform

from keras import Input, Model
from keras.callbacks import EarlyStopping
from keras.constraints import non_neg, max_norm
from numpy import array, mean, ones
from pandas import concat
from pandas import DataFrame
from keras.models import Sequential
from keras.layers import LSTM, multiply, subtract, add, Activation, Lambda
from keras.layers import Dense, concatenate, MaxPooling1D, LocallyConnected1D, Reshape
from keras import backend as K
from keras import constraints

from utils import generate_student_name
import random

nq = 12
n_traits = 5
tr_len = 10
tt_len = 10


Using TensorFlow backend.
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
generate_student_name()

'RAB WUN '

In [4]:
class Question():
    def __init__(self, qix, nt=10):
        #self.MAX_BETA = 15
        self.id = qix
        self.qvals = [ int(uniform(0,1)>=0.5) for _ in range(nt) ]
        #self.betas= [ (qval*randint(1,self.MAX_BETA)) for qval in qvals ]
        #print("Made q with betas:", self.betas)

random.seed(666)
qs = [Question(qix, n_traits) for qix in range(10)]
for q in qs:
    print(q.id, q.qvals)

0 [0, 1, 0, 1, 1]
1 [1, 1, 0, 1, 0]
2 [1, 1, 1, 0, 1]
3 [0, 1, 0, 1, 1]
4 [0, 1, 1, 0, 1]
5 [0, 1, 0, 1, 1]
6 [0, 1, 1, 0, 0]
7 [1, 0, 0, 0, 0]
8 [0, 1, 1, 1, 1]
9 [1, 1, 1, 0, 1]


In [5]:
def generate_learning_rates(nt=10):
    gammas = [rabdint(1,3) for _ in range(nt)]
    betas= [ randint(1,10) for _ in range(nt) ]
    return betas, gammas

In [6]:
def generate_student_struct(nt=10, nq=20):
    name = generate_student_name()
    alpha = randint(1,10)
    h_practice = [ 0 for _ in range(nt) ]
    o_practice = [ 0 for _ in range(nq) ]
    mastery = [0 for _ in range(nq)]
    
    dict = {}
    dict["name"] = name
    dict["alpha"] = alpha
    dict["h_practice"] = h_practice
    dict["o_practice"] = o_practice
    dict["mastery"] = mastery
    return dict

In [7]:
def attempt_q(student, q: Question, betas, gammas):
    a = student["alpha"]
    Ts = student["h_practice"]
    p = calculate_pass_probability(betas, gammas, a, Ts, q.qvals)
    this_att = uniform(0,1)
    if (this_att <= p):
        passed=True
        student["mastery"][q.id] = 1
    else:
        passed=False

    student["o_practice"][q.id] += 1 
    
    for trix, qv in enumerate(q.qvals):
        if qv==1:
            student["h_practice"][trix] += 1
    return passed

In [8]:
def calculate_pass_probability(betas, gammas, a, Ts, qvals):
    # additive factors model is:
    # p_pass = 1 / 1 + exp(-z)
    # where z = a + sum[1:n]( -b + gT )
 
    component_terms = [ ((g*T)-b) for b,g,T,qv in zip(betas,gammas,Ts,qvals) if qv==1]
    print("calc/ing pass: alpha={}, sum_comps={}".format(a,sum(component_terms)))
    z = a + sum(component_terms)
    print("z={}".format(z))
    try:
        p_pass = 1.0 / (1.0 + exp(-z))
        print("p_pass={}".format(p_pass))
    except OverflowError:
        p_pass = 0.0
    #print("real p_pass = {}".format(p_pass))
    return p_pass
    

In [22]:
from keras import backend as K
from keras.engine.topology import Layer

class ProductLayer(Layer):

    def __init__(self, output_dim, kernel_constraint=None, **kwargs):
        self.output_dim = output_dim
        super(ProductLayer, self).__init__(**kwargs)
        self.kernel_constraint = constraints.get(kernel_constraint)
        

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(1, self.output_dim),
                                      initializer='uniform',
                                      trainable=True,
                                      constraint=self.kernel_constraint)
        
        super(ProductLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        print("sk",self.kernel.shape)
        print("x", x.shape)
        z = K.concatenate([x,self.kernel], axis=0)
        print("internal z shape:", z.shape)
        p = K.prod(z, axis=0)
        print("internal output shape:", p.shape)
        return p

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

In [ ]:
def alpha_predictor(nq):
    x = Input(shape=(nq,), name="in_x")
    u = Input(shape=(nq,), name="in_u")
    hidden = concatenate([x,u])
#     hidden = Dense(int(1.2*nq), activation="relu")(hidden)
#     hidden = Dense(int(0.7*nq), activation="relu")(hidden)
    alpha_est = Dense(1, activation="sigmoid")(hidden)
    model = Model(inputs=[x,u], outputs=[alpha_est])
    model.compile(optimizer='adam', loss="binary_crossentropy")
    return model

In [15]:
def nn_model(nq=20):
    x = Input(shape=(nq,))
    u = Input(shape=(nq,))
#     hidden = Dense(10, activation="relu")(inp)
    hidden = concatenate([x,u])
    outp = Dense(1, activation="sigmoid")(hidden)
    model = Model(inputs=[x,u], outputs=outp)
    model.compile(optimizer='adam', loss="binary_crossentropy", metrics=["acc"])
    return model

def cmu_nn_model(nq, nt, blayer: ProductLayer, glayer: ProductLayer): #, qlayer: Dense):
    x = Input(shape=(nq,), name="in_x")
    u = Input(shape=(nq,), name="in_u")
#     T = qlayer(x)
    xsum = Lambda(lambda v : K.sum(v, axis=1, keepdims=True), name="xsum")(x)
    usum = Lambda(lambda v : K.sum(v, axis=1, keepdims=True), name="usum")(u)

    T = Dense(nt, activation="relu", name="T_estimate")(concatenate([x,u]))
    a = Dense(1, activation="relu", name="alpha_estimate")(concatenate([x,u]))
    #a = Input(batch_shape=(None,1), name="in_a")
    
#     xsum = Lambda(lambda v : K.sum(v, axis=1, keepdims=True), name="xsum")(x)
#     usum = Lambda(lambda v : K.sum(v, axis=1, keepdims=True), name="usum")(u)
#     print(xsum)
#     a_h = concatenate([u,T])
#     a_h = concatenate([u,T,x,xsum,usum]) #, name="concat_Txu")
#     a_h = concatenate([x, u], name="concat_Txu")
#     a_h = Dense(10, activation="relu")(a_h)
#     a_h = Dense(1, activation="relu", name="alpha")(a_h)
    g_in = Input(batch_shape=(None, nt), name="in_g")
    b_in = Input(batch_shape=(None, nt), name="in_b")
#     g_h = Reshape((nt,1))(g_in)
#     b_h = Reshape((nt,1))(b_in)
    g_l = glayer(g_in)
#     g_l = Reshape((nt,))(g_l)
    b_l = blayer(b_in)
#     b_l = Reshape((nt,))(b_l)
    gT = multiply([T,g_l], name="gT")
    gT_sub_b = subtract([gT,b_l], name="gT_sub_b")
#     sum_components = Dense(1, activation="relu", name="wsum_gT_sub_b")(gT_b)
    sum_components = Lambda(lambda v : K.sum(v, axis=1, keepdims=True), name="sum_components")(gT_sub_b)
    z = add([a,sum_components]) #, name="z")
    z = Lambda(lambda v: -v, name="neg_z")(z)
    Pr = Activation("sigmoid", name="Pr_z")(z)
    model = Model(inputs=[x,u,b_in,g_in], outputs=[Pr])
    model.compile(optimizer='adam', loss=["binary_crossentropy"], loss_weights=[1.0], metrics=["acc"])
    return model

bvar = ProductLayer(n_traits)
gvar = ProductLayer(n_traits)
# qvar = Dense(n_traits, activation="sigmoid", use_bias=False, kernel_constraint=non_neg(), name="qmatrix")

cmu_nn_model(nq, n_traits, bvar, gvar) #, qvar)
    

sk (1, 5)
x (?, 5)
internal z shape: (?, 5)
internal output shape: (5,)
sk (1, 5)
x (?, 5)
internal z shape: (?, 5)
internal output shape: (5,)


In [16]:
betas, gammas = generate_learning_rates(n_traits)
print("betas=",betas)
print("gammas=",gammas)

master_qs = [Question(qix, n_traits) for qix in range(nq)]
for q in master_qs:
    nocomps = sum(q.qvals)
    mag = sqrt(sum([ qmxv*pow(b, 2) for qmxv,b in zip(q.qvals,betas) ])) 
    print("Q:{}, difficulty={:.2f} across {} components".format(q.id, mag, nocomps))

betas= [9, 3, 7, 7, 8]
gammas= [1, 1, 2, 3, 3]
Q:0, difficulty=11.40 across 2 components
Q:1, difficulty=12.73 across 3 components
Q:2, difficulty=11.40 across 2 components
Q:3, difficulty=13.93 across 3 components
Q:4, difficulty=12.04 across 2 components
Q:5, difficulty=11.05 across 3 components
Q:6, difficulty=11.79 across 3 components
Q:7, difficulty=13.93 across 3 components
Q:8, difficulty=11.40 across 2 components
Q:9, difficulty=15.59 across 4 components
Q:10, difficulty=15.59 across 4 components
Q:11, difficulty=12.04 across 2 components


# Training
This is where sh!t gets real.  We take our tr_len (1000?) students, and iterate over them 100 times to create 100,000 *complete examples* of a student attacking the curriculum.  The questions themselves are attacked in random order: the student has no intelligent guidance throught the material. (Obvious we may wish to provide that guidance at some point in the future.)

Remember, there are only 12 exercises in the curriculum, so if the student is taking 60 or 70 goes to answer them all, that's pretty poor.  But some of these kids are dumb as lumber, so cut them some slack!  They will all get there in the end since by the CMU AFM practice will, eventually, make perfect!

In [43]:
Xs = {}
Tz = {}
ys = {}
psi_list = []
num_tr_students=1000
for _ in range(num_tr_students):
    psi = generate_student_struct(n_traits, nq)
#     print(psi)
    psi_list.append(psi)

for run in range(1):
    print("----{}\n".format(run))
    for psi in psi_list:
        ct = 0
        psi["h_practice"] = [0 for _ in range(n_traits)]
        psi["o_practice"] = [0 for _ in range(nq)]
        psi["mastery"] = [0 for _ in range(nq)]
        qs = copy(master_qs)
        while(True):
            q = random.choice(qs)
            qix = q.id
            if qix not in Xs:
                Xs[qix]=[]
                Tz[qix]=[]
                ys[qix]=[]

            pre_question_profile = (tuple(psi["o_practice"]), tuple(psi["mastery"]))
#             print("PQP {} {}: {}".format(psi["name"], qix, pre_question_profile))
            Xs[qix].append( pre_question_profile )
            Tz[qix].append(tuple(psi["h_practice"]))
            
            #print("attempting {}".format(q.id))
            passed = attempt_q(psi, q, betas, gammas)
            ys[qix].append((1.0 if passed else 0.0, psi["alpha"]))

            ct+=1
            if passed:
                #print("passed")
                qs.remove(q)

            if qs == []:
                break
        print("{} ({}): curriculum completed in {} attempts".format(psi["name"], psi["alpha"], ct))

----0

calc/ing pass: alpha=3, sum_comps=-31
z=-28
p_pass=6.914400106935423e-13
calc/ing pass: alpha=3, sum_comps=-22
z=-19
p_pass=5.602796406145941e-09
calc/ing pass: alpha=3, sum_comps=-6
z=-3
p_pass=0.04742587317756678
calc/ing pass: alpha=3, sum_comps=2
z=5
p_pass=0.9933071490757153
calc/ing pass: alpha=3, sum_comps=-3
z=0
p_pass=0.5
calc/ing pass: alpha=3, sum_comps=1
z=4
p_pass=0.9820137900379085
calc/ing pass: alpha=3, sum_comps=0
z=3
p_pass=0.9525741268224334
calc/ing pass: alpha=3, sum_comps=-3
z=0
p_pass=0.5
calc/ing pass: alpha=3, sum_comps=0
z=3
p_pass=0.9525741268224334
calc/ing pass: alpha=3, sum_comps=3
z=6
p_pass=0.9975273768433653
calc/ing pass: alpha=3, sum_comps=25
z=28
p_pass=0.9999999999993086
calc/ing pass: alpha=3, sum_comps=13
z=16
p_pass=0.9999998874648379
calc/ing pass: alpha=3, sum_comps=24
z=27
p_pass=0.9999999999981204
calc/ing pass: alpha=3, sum_comps=44
z=47
p_pass=1.0
calc/ing pass: alpha=3, sum_comps=36
z=39
p_pass=1.0
calc/ing pass: alpha=3, sum_comps=

p_pass=0.00012339457598623172
calc/ing pass: alpha=5, sum_comps=-10
z=-5
p_pass=0.0066928509242848554
calc/ing pass: alpha=5, sum_comps=-6
z=-1
p_pass=0.2689414213699951
calc/ing pass: alpha=5, sum_comps=-9
z=-4
p_pass=0.01798620996209156
calc/ing pass: alpha=5, sum_comps=-2
z=3
p_pass=0.9525741268224334
calc/ing pass: alpha=5, sum_comps=-3
z=2
p_pass=0.8807970779778823
calc/ing pass: alpha=5, sum_comps=0
z=5
p_pass=0.9933071490757153
calc/ing pass: alpha=5, sum_comps=10
z=15
p_pass=0.999999694097773
calc/ing pass: alpha=5, sum_comps=6
z=11
p_pass=0.999983298578152
calc/ing pass: alpha=5, sum_comps=18
z=23
p_pass=0.9999999998973812
calc/ing pass: alpha=5, sum_comps=14
z=19
p_pass=0.9999999943972036
calc/ing pass: alpha=5, sum_comps=29
z=34
p_pass=0.9999999999999982
calc/ing pass: alpha=5, sum_comps=39
z=44
p_pass=1.0
calc/ing pass: alpha=5, sum_comps=27
z=32
p_pass=0.9999999999999873
calc/ing pass: alpha=5, sum_comps=12
z=17
p_pass=0.9999999586006244
calc/ing pass: alpha=5, sum_comps=1

calc/ing pass: alpha=4, sum_comps=17
z=21
p_pass=0.9999999992417439
calc/ing pass: alpha=4, sum_comps=23
z=27
p_pass=0.9999999999981204
calc/ing pass: alpha=4, sum_comps=40
z=44
p_pass=1.0
calc/ing pass: alpha=4, sum_comps=22
z=26
p_pass=0.999999999994891
NYSS CHYHI  (4): curriculum completed in 18 attempts
calc/ing pass: alpha=3, sum_comps=-16
z=-13
p_pass=2.2603242979035746e-06
calc/ing pass: alpha=3, sum_comps=-15
z=-12
p_pass=6.144174602214718e-06
calc/ing pass: alpha=3, sum_comps=-24
z=-21
p_pass=7.582560422162385e-10
calc/ing pass: alpha=3, sum_comps=-9
z=-6
p_pass=0.0024726231566347743
calc/ing pass: alpha=3, sum_comps=-9
z=-6
p_pass=0.0024726231566347743
calc/ing pass: alpha=3, sum_comps=-9
z=-6
p_pass=0.0024726231566347743
calc/ing pass: alpha=3, sum_comps=-6
z=-3
p_pass=0.04742587317756678
calc/ing pass: alpha=3, sum_comps=0
z=3
p_pass=0.9525741268224334
calc/ing pass: alpha=3, sum_comps=9
z=12
p_pass=0.9999938558253978
calc/ing pass: alpha=3, sum_comps=4
z=7
p_pass=0.9990889

p_pass=0.9999999999997455
calc/ing pass: alpha=9, sum_comps=34
z=43
p_pass=1.0
GYLL LAJU  (9): curriculum completed in 17 attempts
calc/ing pass: alpha=3, sum_comps=-22
z=-19
p_pass=5.602796406145941e-09
calc/ing pass: alpha=3, sum_comps=-14
z=-11
p_pass=1.670142184809518e-05
calc/ing pass: alpha=3, sum_comps=-12
z=-9
p_pass=0.00012339457598623172
calc/ing pass: alpha=3, sum_comps=-14
z=-11
p_pass=1.670142184809518e-05
calc/ing pass: alpha=3, sum_comps=-5
z=-2
p_pass=0.11920292202211755
calc/ing pass: alpha=3, sum_comps=-1
z=2
p_pass=0.8807970779778823
calc/ing pass: alpha=3, sum_comps=-5
z=-2
p_pass=0.11920292202211755
calc/ing pass: alpha=3, sum_comps=6
z=9
p_pass=0.9998766054240137
calc/ing pass: alpha=3, sum_comps=7
z=10
p_pass=0.9999546021312976
calc/ing pass: alpha=3, sum_comps=10
z=13
p_pass=0.999997739675702
calc/ing pass: alpha=3, sum_comps=21
z=24
p_pass=0.9999999999622486
calc/ing pass: alpha=3, sum_comps=22
z=25
p_pass=0.999999999986112
calc/ing pass: alpha=3, sum_comps=6
z

z=14
p_pass=0.9999991684719722
calc/ing pass: alpha=4, sum_comps=13
z=17
p_pass=0.9999999586006244
calc/ing pass: alpha=4, sum_comps=38
z=42
p_pass=1.0
calc/ing pass: alpha=4, sum_comps=11
z=15
p_pass=0.999999694097773
calc/ing pass: alpha=4, sum_comps=20
z=24
p_pass=0.9999999999622486
calc/ing pass: alpha=4, sum_comps=15
z=19
p_pass=0.9999999943972036
calc/ing pass: alpha=4, sum_comps=24
z=28
p_pass=0.9999999999993086
CIDY TYLU  (4): curriculum completed in 18 attempts
calc/ing pass: alpha=6, sum_comps=-24
z=-18
p_pass=1.522997951276035e-08
calc/ing pass: alpha=6, sum_comps=-13
z=-7
p_pass=0.0009110511944006454
calc/ing pass: alpha=6, sum_comps=-14
z=-8
p_pass=0.0003353501304664781
calc/ing pass: alpha=6, sum_comps=-11
z=-5
p_pass=0.0066928509242848554
calc/ing pass: alpha=6, sum_comps=-3
z=3
p_pass=0.9525741268224334
calc/ing pass: alpha=6, sum_comps=3
z=9
p_pass=0.9998766054240137
calc/ing pass: alpha=6, sum_comps=1
z=7
p_pass=0.9990889488055994
calc/ing pass: alpha=6, sum_comps=5
z

...

calc/ing pass: alpha=2, sum_comps=36
z=38
p_pass=1.0
FIVED DYPHEM  (2): curriculum completed in 20 attempts
calc/ing pass: alpha=1, sum_comps=-16
z=-15
p_pass=3.059022269256247e-07
calc/ing pass: alpha=1, sum_comps=-18
z=-17
p_pass=4.1399375473943306e-08
calc/ing pass: alpha=1, sum_comps=-15
z=-14
p_pass=8.315280276641321e-07
calc/ing pass: alpha=1, sum_comps=-11
z=-10
p_pass=4.5397868702434395e-05
calc/ing pass: alpha=1, sum_comps=-7
z=-6
p_pass=0.0024726231566347743
calc/ing pass: alpha=1, sum_comps=-8
z=-7
p_pass=0.0009110511944006454
calc/ing pass: alpha=1, sum_comps=-6
z=-5
p_pass=0.0066928509242848554
calc/ing pass: alpha=1, sum_comps=4
z=5
p_pass=0.9933071490757153
calc/ing pass: alpha=1, sum_comps=9
z=10
p_pass=0.9999546021312976
calc/ing pass: alpha=1, sum_comps=-3
z=-2
p_pass=0.11920292202211755
calc/ing pass: alpha=1, sum_comps=14
z=15
p_pass=0.999999694097773
calc/ing pass: alpha=1, sum_comps=21
z=22
p_pass=0.9999999997210531
calc/ing pass: alpha=1, sum_comps=23
z=24
p_pass

p_pass=5.602796406145941e-09
calc/ing pass: alpha=3, sum_comps=-6
z=-3
p_pass=0.04742587317756678
calc/ing pass: alpha=3, sum_comps=-10
z=-7
p_pass=0.0009110511944006454
calc/ing pass: alpha=3, sum_comps=-5
z=-2
p_pass=0.11920292202211755
calc/ing pass: alpha=3, sum_comps=5
z=8
p_pass=0.9996646498695336
calc/ing pass: alpha=3, sum_comps=-2
z=1
p_pass=0.7310585786300049
calc/ing pass: alpha=3, sum_comps=-3
z=0
p_pass=0.5
calc/ing pass: alpha=3, sum_comps=5
z=8
p_pass=0.9996646498695336
calc/ing pass: alpha=3, sum_comps=5
z=8
p_pass=0.9996646498695336
calc/ing pass: alpha=3, sum_comps=2
z=5
p_pass=0.9933071490757153
calc/ing pass: alpha=3, sum_comps=26
z=29
p_pass=0.9999999999997455
calc/ing pass: alpha=3, sum_comps=21
z=24
p_pass=0.9999999999622486
calc/ing pass: alpha=3, sum_comps=27
z=30
p_pass=0.9999999999999065
calc/ing pass: alpha=3, sum_comps=32
z=35
p_pass=0.9999999999999993
calc/ing pass: alpha=3, sum_comps=53
z=56
p_pass=1.0
calc/ing pass: alpha=3, sum_comps=29
z=32
p_pass=0.99

z=-8
p_pass=0.0003353501304664781
calc/ing pass: alpha=7, sum_comps=-10
z=-3
p_pass=0.04742587317756678
calc/ing pass: alpha=7, sum_comps=-14
z=-7
p_pass=0.0009110511944006454
calc/ing pass: alpha=7, sum_comps=-5
z=2
p_pass=0.8807970779778823
calc/ing pass: alpha=7, sum_comps=-11
z=-4
p_pass=0.01798620996209156
calc/ing pass: alpha=7, sum_comps=-2
z=5
p_pass=0.9933071490757153
calc/ing pass: alpha=7, sum_comps=2
z=9
p_pass=0.9998766054240137
calc/ing pass: alpha=7, sum_comps=2
z=9
p_pass=0.9998766054240137
calc/ing pass: alpha=7, sum_comps=15
z=22
p_pass=0.9999999997210531
calc/ing pass: alpha=7, sum_comps=0
z=7
p_pass=0.9990889488055994
calc/ing pass: alpha=7, sum_comps=23
z=30
p_pass=0.9999999999999065
calc/ing pass: alpha=7, sum_comps=14
z=21
p_pass=0.9999999992417439
calc/ing pass: alpha=7, sum_comps=17
z=24
p_pass=0.9999999999622486
calc/ing pass: alpha=7, sum_comps=24
z=31
p_pass=0.9999999999999656
calc/ing pass: alpha=7, sum_comps=30
z=37
p_pass=1.0
calc/ing pass: alpha=7, sum_c

calc/ing pass: alpha=7, sum_comps=50
z=57
p_pass=1.0
DUJY CHICO  (7): curriculum completed in 16 attempts
calc/ing pass: alpha=8, sum_comps=-24
z=-16
p_pass=1.12535162055095e-07
calc/ing pass: alpha=8, sum_comps=-18
z=-10
p_pass=4.5397868702434395e-05
calc/ing pass: alpha=8, sum_comps=-12
z=-4
p_pass=0.01798620996209156
calc/ing pass: alpha=8, sum_comps=-16
z=-8
p_pass=0.0003353501304664781
calc/ing pass: alpha=8, sum_comps=-6
z=2
p_pass=0.8807970779778823
calc/ing pass: alpha=8, sum_comps=-6
z=2
p_pass=0.8807970779778823
calc/ing pass: alpha=8, sum_comps=3
z=11
p_pass=0.999983298578152
calc/ing pass: alpha=8, sum_comps=10
z=18
p_pass=0.9999999847700205
calc/ing pass: alpha=8, sum_comps=6
z=14
p_pass=0.9999991684719722
calc/ing pass: alpha=8, sum_comps=21
z=29
p_pass=0.9999999999997455
calc/ing pass: alpha=8, sum_comps=16
z=24
p_pass=0.9999999999622486
calc/ing pass: alpha=8, sum_comps=27
z=35
p_pass=0.9999999999999993
calc/ing pass: alpha=8, sum_comps=41
z=49
p_pass=1.0
calc/ing pass:

calc/ing pass: alpha=3, sum_comps=25
z=28
p_pass=0.9999999999993086
calc/ing pass: alpha=3, sum_comps=49
z=52
p_pass=1.0
VOOTU NOOPO  (3): curriculum completed in 18 attempts
calc/ing pass: alpha=4, sum_comps=-16
z=-12
p_pass=6.144174602214718e-06
calc/ing pass: alpha=4, sum_comps=-23
z=-19
p_pass=5.602796406145941e-09
calc/ing pass: alpha=4, sum_comps=-17
z=-13
p_pass=2.2603242979035746e-06
calc/ing pass: alpha=4, sum_comps=-9
z=-5
p_pass=0.0066928509242848554
calc/ing pass: alpha=4, sum_comps=-7
z=-3
p_pass=0.04742587317756678
calc/ing pass: alpha=4, sum_comps=-5
z=-1
p_pass=0.2689414213699951
calc/ing pass: alpha=4, sum_comps=-2
z=2
p_pass=0.8807970779778823
calc/ing pass: alpha=4, sum_comps=1
z=5
p_pass=0.9933071490757153
calc/ing pass: alpha=4, sum_comps=-8
z=-4
p_pass=0.01798620996209156
calc/ing pass: alpha=4, sum_comps=-1
z=3
p_pass=0.9525741268224334
calc/ing pass: alpha=4, sum_comps=1
z=5
p_pass=0.9933071490757153
calc/ing pass: alpha=4, sum_comps=15
z=19
p_pass=0.99999999439

In [44]:
for qix in Xs.keys():
    print("question",qix)
    x,u =zip(*Xs[qix])
    x = array(x)
    u = array(u)
    t = array(Tz[qix])
    y,ya = zip(*ys[qix])
    y= array(y)
    ya= array(ya)
    
    for ix in range(10):
        print(x[ix,:], u[ix,:], t[ix,:], y[ix], ya[ix])
    print(x.shape)

question 10
[0 0 0 0 0 0 0 0 0 0 0 0] [0 0 0 0 0 0 0 0 0 0 0 0] [0 0 0 0 0] 0.0 3
[1 2 1 0 2 0 1 0 1 1 1 0] [1 1 1 0 1 0 1 0 1 0 0 0] [8 1 6 6 6] 1.0 3
[2 0 0 2 1 1 0 0 1 0 0 1] [1 0 0 0 0 0 0 0 0 0 0 1] [7 1 3 3 5] 1.0 8
[3 1 0 1 2 1 0 2 0 1 0 3] [1 1 0 1 1 1 0 1 0 1 0 1] [12  1  5  6 11] 1.0 5
[1 1 1 2 1 1 1 1 3 1 0 2] [0 1 0 1 1 1 0 1 1 1 0 1] [13  2  9  5  9] 1.0 4
[0 0 0 0 0 0 0 0 0 0 0 0] [0 0 0 0 0 0 0 0 0 0 0 0] [0 0 0 0 0] 0.0 1
[0 1 0 1 0 0 0 0 0 1 1 1] [0 0 0 0 0 0 0 0 0 0 0 0] [4 0 4 3 5] 1.0 1
[0 2 0 0 0 0 0 0 1 0 0 0] [0 0 0 0 0 0 0 0 0 0 0 0] [1 0 3 2 2] 0.0 4
[1 2 1 1 1 1 1 1 1 1 1 0] [1 0 0 1 1 1 1 1 0 1 0 0] [9 2 8 7 8] 1.0 4
[1 2 2 1 1 1 1 2 1 1 0 1] [1 1 1 1 1 1 1 1 1 1 0 1] [11  2  9  6  9] 1.0 10
(1424, 12)
question 9
[0 0 0 0 0 0 0 0 0 0 1 0] [0 0 0 0 0 0 0 0 0 0 0 0] [1 0 1 1 1] 0.0 3
[1 2 1 1 2 0 1 0 1 1 2 1] [1 1 1 1 1 0 1 0 1 0 1 1] [11  1  8  7  9] 1.0 3
[2 1 1 2 1 1 1 1 1 0 1 1] [1 1 1 0 0 0 1 1 0 0 1 1] [11  2  7  6  8] 1.0 8
[2 0 0 0 2 0 0 2 0 0 0 2] [0 0

In [45]:
alpha_model = alpha_predictor(nq)
xs = []
us = []
tas = []
for qix in Xs.keys():
    print("question",qix)
    x,u = zip(*Xs[qix])
    y,true_a = zip(*ys[qix])
    xs += x
    us += u
    tas += y

es = EarlyStopping(monitor="loss", mode="auto")
for ep in range(30):
    xs = array(xs)
    us = array(us)
    tas = array(tas)
    alpha_model.fit([xs,us],[tas], verbose=1, epochs=30, shuffle=True, callbacks=[es])
    a_pds = list(alpha_model.predict([xs,us]))
    score = alpha_model.evaluate([xs,us], [tas])
    print(score)
    
# for x,u,ta, pa in zip(xs,us,tas, a_pds):
#      print(x,u,ta,pa)

NameError: name 'alpha_predictor' is not defined

In [46]:
bvar = ProductLayer(n_traits, name="betas", kernel_constraint=non_neg())
gvar = ProductLayer(n_traits, name="gammas", kernel_constraint=non_neg())
#qvar = Dense(n_traits, activation="sigmoid", use_bias=False, kernel_constraint=max_norm(sqrt(n_traits), axis=1), name="qmatrix")

models = [cmu_nn_model(nq, n_traits, bvar, gvar) for _ in range(nq)]


from keras.utils import plot_model
plot_model(models[0], to_file='cmu_sim_model.png')
models[0].summary()
input("go")

for qix in Xs.keys():
    print("question",qix)
    x,u =zip(*Xs[qix])
    x = array(x)
#     a = ones((x.shape[0],1))
    b = ones((x.shape[0],n_traits))
    g = ones((x.shape[0],n_traits))
    u = array(u)
#     T = array(Tz[qix])
    y,true_a = zip(*ys[qix])
    y = array(y)
#     true_a = array(true_a)
    es = EarlyStopping(patience=0, monitor="val_loss", mode="auto")
#     models[qix].fit([x,u],y, verbose=0, shuffle=True, epochs=1000, validation_split=0.1, callbacks=[es])
#     score = models[qix].evaluate([x,u],y)
#     for xx,uu in zip(x,u):
#         print(xx,uu)
#     input("now predict...")
#     y_hat, a_hat = models[qix].predict([a,b,g,x,u])
#     for yh,yy, ah,aa in zip(y_hat,y,a_hat,true_a):
#         print(yh,yy,ah,aa)
#     input("next....")
    models[qix].fit([x,u,b,g],[y], verbose=0, shuffle=True, epochs=1000, validation_split=0.1, callbacks=[es])
#     print("{}, a hat and true_a = {} v {}".format(y_hat, a_hat, true_a))
    score = models[qix].evaluate([x,u,b,g],[y])
    print(score)

sk (1, 5)
x (?, 5)
internal z shape: (?, 5)
internal output shape: (5,)
sk (1, 5)
x (?, 5)
internal z shape: (?, 5)
internal output shape: (5,)
sk (1, 5)
x (?, 5)
internal z shape: (?, 5)
internal output shape: (5,)
sk (1, 5)
x (?, 5)
internal z shape: (?, 5)
internal output shape: (5,)
sk (1, 5)
x (?, 5)
internal z shape: (?, 5)
internal output shape: (5,)
sk (1, 5)
x (?, 5)
internal z shape: (?, 5)
internal output shape: (5,)
sk (1, 5)
x (?, 5)
internal z shape: (?, 5)
internal output shape: (5,)
sk (1, 5)
x (?, 5)
internal z shape: (?, 5)
internal output shape: (5,)
sk (1, 5)
x (?, 5)
internal z shape: (?, 5)
internal output shape: (5,)
sk (1, 5)
x (?, 5)
internal z shape: (?, 5)
internal output shape: (5,)
sk (1, 5)
x (?, 5)
internal z shape: (?, 5)
internal output shape: (5,)
sk (1, 5)
x (?, 5)
internal z shape: (?, 5)
internal output shape: (5,)
sk (1, 5)
x (?, 5)
internal z shape: (?, 5)
internal output shape: (5,)
sk (1, 5)
x (?, 5)
internal z shape: (?, 5)
internal output shap

In [47]:
# bz = b_layer0.get_weights()
# gz = g_layer0.get_weights()
# print(bz)
# print(gz)
# gconfig = g_layer0.get_config()
# print(models[0].get_weights())
# for layer in models[0].layers: print(layer.get_weights())
# print("\n\nXXXXXX\n\n")
# for layer in models[1].layers: print(layer.get_weights())
print("bvar")
print(bvar.get_weights())
print(betas)
print("gvar")
print(gvar.get_weights())
print(gammas)
print("qvar")
print(qvar.get_weights())

bvar
[array([[ 0.69880581,  0.69880581,  0.71598822,  0.69880581,  0.70786023]], dtype=float32)]
[9, 3, 7, 7, 8]
gvar
[array([[ 3.94856501,  5.60812712,  4.98467922,  3.74015903,  4.13600922]], dtype=float32)]
[1, 1, 2, 3, 3]
qvar
[]


In [48]:
psi_scores = defaultdict(int)
psi_cts = defaultdict(int)
psi_devs = defaultdict(list)
# psi_alphas = defaultdict(list)

psi_test_list = []
for _ in range(10):
    psi = generate_student_struct()
    print(psi)
    psi_test_list.append(psi)

for psi in psi_test_list:
    psi_name = psi["name"]
    psi["h_practice"] = [0 for _ in range(n_traits)]
    psi["o_practice"] = [0 for _ in range(nq)]
    psi["mastery"] = [0 for _ in range(nq)]
    qs = copy(master_qs)
    match_ct=0
    ct=0
    kcts = [0 for i in range(nq)]
    while(True):
        q = random.choice(qs)
        qix = q.id
        print("chose q{}".format(qix))

        x = array(psi["o_practice"]).reshape(1,-1)
#         h = psi["h_practice"]
#         alpha = psi["alpha"]
        u = array(psi["mastery"]).reshape(1,-1)
#         a = ones((1,1))
        b = ones((1,n_traits))
        g = ones((1,n_traits))
    
#         y_hat, y_alpha = models[qix].predict([a,b,g,x,u]) #.reshape(1,-1))
        y_hat = models[qix].predict([x,u,b,g]) #.reshape(1,-1))
#         alpha_pred = alpha_model.predict([x,u])
#         psi_alphas[psi_name].append(alpha_pred)
    
        ytrue = calculate_pass_probability(betas, gammas, alpha, h, q.qvals)
        passed = attempt_q(psi, q, betas, gammas)
        print("yhp",y_hat,passed, ytrue)
        if (y_hat>=0.5) == passed:
            match_ct += 1
            psi_scores[psi_name]+=1
            print("MATCH GOOD")
        
            
#         print("Alpha comp:", y_alpha, psi["alpha"])
#         psi_devs[psi_name].append((y_alpha - psi["alpha"]))
        psi_cts[psi_name]+=1
        ct+=1
        kcts[qix]+=1
        if passed:
            print("passed")
            qs.remove(q)
        else:
            print("failed")

        psi["o_practice"][qix]+=1
        print("psi: {}, IQ: {}".format(psi["name"],psi["alpha"]))
        print("practice is now:", psi["o_practice"], psi["h_practice"])
        print("mastery is now:", psi["mastery"])

        if qs == []:
            print("complete")
            break
    print("curriculum completed in {} attempts".format(ct))
    print("skill-level counts:")
    print(kcts)
    print("{}/{}".format(match_ct, ct))

{'name': 'SSUTHYCH KOTOK ', 'alpha': 6, 'h_practice': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'o_practice': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'mastery': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
{'name': 'SUJAPH KUTI ', 'alpha': 1, 'h_practice': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'o_practice': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'mastery': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
{'name': 'WIVOO TOS ', 'alpha': 3, 'h_practice': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'o_practice': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'mastery': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
{'name': 'PHYJYH SSEW ', 'alpha': 3, 'h_practice': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'o_practice': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'mastery': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
{'name': 'FEPA THIGI ', 'alpha': 10, 'h_practice': [0, 0, 0, 0, 0, 0, 0

calc/ing pass: alpha=4, sum_comps=32
z=36
p_pass=0.9999999999999998
calc/ing pass: alpha=10, sum_comps=-9
z=1
p_pass=0.7310585786300049
yhp [[ 0.94428504]] True 0.9999999999999998
MATCH GOOD
passed
psi: FEPA THIGI , IQ: 10
practice is now: [0, 2, 0, 0, 2, 0, 0, 0, 2, 0, 0, 2] [3, 0, 2, 1, 3]
mastery is now: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1]
chose q1
calc/ing pass: alpha=4, sum_comps=61
z=65
p_pass=1.0
calc/ing pass: alpha=10, sum_comps=-6
z=4
p_pass=0.9820137900379085
yhp [[ 0.69669485]] True 1.0
MATCH GOOD
passed
psi: FEPA THIGI , IQ: 10
practice is now: [0, 4, 0, 0, 2, 0, 0, 0, 2, 0, 0, 2] [3, 0, 3, 2, 4]
mastery is now: [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1]
chose q3
calc/ing pass: alpha=4, sum_comps=45
z=49
p_pass=1.0
calc/ing pass: alpha=10, sum_comps=-3
z=7
p_pass=0.9990889488055994
yhp [[ 0.97125894]] True 1.0
MATCH GOOD
passed
psi: FEPA THIGI , IQ: 10
practice is now: [0, 4, 0, 2, 2, 0, 0, 0, 2, 0, 0, 2] [4, 0, 4, 2, 5]
mastery is now: [0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1]
chos

calc/ing pass: alpha=4, sum_comps=17
z=21
p_pass=0.9999999992417439
calc/ing pass: alpha=2, sum_comps=-2
z=0
p_pass=0.5
yhp [[ 0.7877683]] True 0.9999999992417439
MATCH GOOD
passed
psi: JUCEW RAC , IQ: 2
practice is now: [0, 2, 2, 0, 0, 0, 2, 2, 0, 0, 4, 4] [7, 1, 5, 4, 6]
mastery is now: [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1]
chose q8
calc/ing pass: alpha=4, sum_comps=17
z=21
p_pass=0.9999999992417439
calc/ing pass: alpha=2, sum_comps=1
z=3
p_pass=0.9525741268224334
yhp [[ 0.73997009]] True 0.9999999992417439
MATCH GOOD
passed
psi: JUCEW RAC , IQ: 2
practice is now: [0, 2, 2, 0, 0, 0, 2, 2, 2, 0, 4, 4] [8, 1, 6, 4, 6]
mastery is now: [0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1]
chose q10
calc/ing pass: alpha=4, sum_comps=65
z=69
p_pass=1.0
calc/ing pass: alpha=2, sum_comps=19
z=21
p_pass=0.9999999992417439
yhp [[  1.15233041e-08]] True 1.0
passed
psi: JUCEW RAC , IQ: 2
practice is now: [0, 2, 2, 0, 0, 0, 2, 2, 2, 0, 6, 4] [9, 1, 7, 5, 7]
mastery is now: [0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1]
chos

In [49]:
for psi_name in sorted(list(psi_scores)):
    print("Scores for {}:".format(psi_name))
    print("{}/{} =\t{}".format(psi_scores[psi_name], psi_cts[psi_name],(psi_scores[psi_name]/psi_cts[psi_name])))
#     print("mean alpha pred = {}".format(mean(psi_alphas[psi_name])))
#     print("Alpha Mae: {}".format(  mean([abs(d) for d in psi_devs[psi_name]]) ))
    

Scores for CAD LORUB :
18/19 =	0.9473684210526315
Scores for FEPA THIGI :
12/14 =	0.8571428571428571
Scores for JUCEW RAC :
15/17 =	0.8823529411764706
Scores for PABOOCH LOOJA :
13/16 =	0.8125
Scores for PHYJYH SSEW :
16/18 =	0.8888888888888888
Scores for SSOOFEC RYSS :
17/19 =	0.8947368421052632
Scores for SSUTHYCH KOTOK :
13/16 =	0.8125
Scores for SUJAPH KUTI :
19/20 =	0.95
Scores for WIVOO TOS :
14/17 =	0.8235294117647058
Scores for YOOB THOLLEG :
14/17 =	0.8235294117647058
